In [1]:
# 1. 환경 설정
!pip install -q -U torch transformers peft bitsandbytes accelerate pandas datasets trl evaluate rouge_score

In [2]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, SFTConfig

# Set random seed
def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/conda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Using device: cuda


## 2. 개선된 설정 (v2)

In [3]:
# ============================================
# 개선된 학습 설정 v2 (현실적 최적화)
# ============================================
# v1 학습시간 기준: ~30분 (epoch=1, batch=2, seq=1024)
# v2 예상 학습시간: ~60-90분 (epoch=2, batch=1, seq=1024)
# ============================================
CONF = {
    "model_name": "upstage/SOLAR-10.7B-Instruct-v1.0",
    "data_path": "./data/",
    "output_dir": "./results_solar_v2",
    
    # LoRA 설정 (개선 - 핵심)
    "lora_r": 16,              # 8 → 16 ⭐ (표현력 2배, +0.3~0.5점 기대)
    "lora_alpha": 64,          # 32 → 64 (scaling 비율 유지)
    "lora_dropout": 0.1,       # 0.05 → 0.1 (과적합 방지)
    
    # 학습 설정 (현실적 최적화)
    "lr": 2e-4,                # 1e-4 → 2e-4 (빠른 수렴)
    "batch_size": 1,           # 2 → 1 (메모리 절약)
    "grad_accum": 8,           # 4 → 8 (effective batch=8 유지)
    "epochs": 2,               # 1 → 2 ⭐ (3은 과적합 위험 + 시간 과다)
    "max_seq_length": 1024,    # 1024 유지 (메모리 안전, 데이터 대부분 커버)
    "warmup_ratio": 0.1,       # 0.03 → 0.1 (안정적 시작)
    "lr_scheduler": "cosine",  # constant → cosine ⭐ (후반 미세조정)
}

print("📋 학습 설정 v2:")
for k, v in CONF.items():
    print(f"  {k}: {v}")

📋 학습 설정 v2:
  model_name: upstage/SOLAR-10.7B-Instruct-v1.0
  data_path: ./data/
  output_dir: ./results_solar_v2
  lora_r: 16
  lora_alpha: 64
  lora_dropout: 0.1
  lr: 0.0002
  batch_size: 1
  grad_accum: 8
  epochs: 2
  max_seq_length: 1024
  warmup_ratio: 0.1
  lr_scheduler: cosine


## 3. 데이터 전처리 (개선)

In [4]:
import re

def clean_text(text):
    """텍스트 정규화 (개선)"""
    # 1. 연속 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 2. 특수 토큰 정규화 - 일관된 형식 유지
    # Person 토큰 (다양한 변형 통일)
    text = re.sub(r'#Person(\d+)#', r'#Person\1#', text)
    
    # 개인정보 토큰 정규화
    text = re.sub(r'#PhoneNumber#', '#PhoneNumber#', text)
    text = re.sub(r'#Address#', '#Address#', text)
    text = re.sub(r'#Email#', '#Email#', text)
    text = re.sub(r'#DateOfBirth#', '#DateOfBirth#', text)
    text = re.sub(r'#CardNumber#', '#CardNumber#', text)
    text = re.sub(r'#CarNumber#', '#CarNumber#', text)
    text = re.sub(r'#PassportNumber#', '#PassportNumber#', text)
    text = re.sub(r'#SSN#', '#SSN#', text)
    
    # 이름 토큰 통일 (#Name#, #PersonName# -> #Name#)
    text = re.sub(r'#PersonName#', '#Name#', text)
    
    return text

def load_data(data_path):
    train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
    dev_df = pd.read_csv(os.path.join(data_path, 'dev.csv'))
    
    # 텍스트 정규화
    train_df['dialogue'] = train_df['dialogue'].apply(clean_text)
    train_df['summary'] = train_df['summary'].apply(clean_text)
    dev_df['dialogue'] = dev_df['dialogue'].apply(clean_text)
    dev_df['summary'] = dev_df['summary'].apply(clean_text)
    
    return train_df, dev_df

def format_instruction(row):
    """SOLAR Instruct 형식 (개선된 프롬프트)"""
    # 개선: 한국어 요약 + 특수 토큰 보존 지시 추가
    prompt = f"""### User:
다음 대화를 한국어로 요약하세요. #Person1#, #Person2# 등의 화자 표시는 그대로 유지하세요.

{row['dialogue']}

### Assistant:
"""
    if 'summary' in row:
        prompt += f"{row['summary']}"
    return prompt

train_df, dev_df = load_data(CONF['data_path'])

# 데이터셋 생성
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

train_dataset = train_dataset.map(lambda x: {'text': format_instruction(x)})
dev_dataset = dev_dataset.map(lambda x: {'text': format_instruction(x)})

print(f"📊 데이터셋 크기:")
print(f"  Train: {len(train_dataset)}")
print(f"  Dev: {len(dev_dataset)}")
print(f"\n📝 샘플:")
print(train_dataset[0]['text'][:500])

Map:   0%|          | 0/12457 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

📊 데이터셋 크기:
  Train: 12457
  Dev: 499

📝 샘플:
### User:
다음 대화를 한국어로 요약하세요. #Person1#, #Person2# 등의 화자 표시는 그대로 유지하세요.

#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? #Person2#: 건강검진을 받으려고 왔어요. #Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. #Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. #Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요. #Person2#: 알겠습니다. #Person1#: 여기 좀 볼까요. 눈과 귀는 괜찮으시네요. 깊게 숨 한 번 쉬어보세요. Mr. Smith, 담배 피우세요? #Person2#: 네. #Person1#: 담배가 폐암하고 심장병의 주된 원인인 거 아시죠? 끊으셔야 해요. #Person2#: 수백 번 시도했


## 4. 모델 로드 (QLoRA)

In [5]:
# 4-bit Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    CONF['model_name'],
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONF['model_name'], trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ 모델 로드 완료: {CONF['model_name']}")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✅ 모델 로드 완료: upstage/SOLAR-10.7B-Instruct-v1.0


## 5. LoRA 설정 (개선)

In [6]:
# 개선된 LoRA 설정
peft_config = LoraConfig(
    r=CONF['lora_r'],              # 16 (증가)
    lora_alpha=CONF['lora_alpha'], # 64 (증가)
    lora_dropout=CONF['lora_dropout'],  # 0.1 (증가)
    bias="none",
    task_type="CAUSAL_LM",
    # 모든 linear layer 타겟
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

print("\n📊 학습 가능한 파라미터:")
model.print_trainable_parameters()


📊 학습 가능한 파라미터:
trainable params: 62,914,560 || all params: 10,794,438,656 || trainable%: 0.5828


## 6. 학습

In [10]:
# 학습 설정 (TRL 0.25+ 호환)
training_args = SFTConfig(
    output_dir=CONF['output_dir'],
    num_train_epochs=CONF['epochs'],
    per_device_train_batch_size=CONF['batch_size'],
    per_device_eval_batch_size=CONF['batch_size'],
    gradient_accumulation_steps=CONF['grad_accum'],
    optim="paged_adamw_32bit",
    save_steps=200,
    save_total_limit=3,
    logging_steps=10,
    learning_rate=CONF['lr'],
    weight_decay=0.01,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=CONF['warmup_ratio'],
    group_by_length=True,
    lr_scheduler_type=CONF['lr_scheduler'],
    report_to="none",
    gradient_checkpointing=True,
    eval_strategy="steps",
    eval_steps=200,
    max_length=CONF['max_seq_length'],  # max_seq_length → max_length
    packing=False,
    dataset_text_field="text",
)

# Trainer 초기화 (TRL 0.25+)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=training_args,
)

print("🚀 학습 시작...")
trainer.train()

# 모델 저장
trainer.model.save_pretrained(CONF['output_dir'] + "/best_model")
tokenizer.save_pretrained(CONF['output_dir'] + "/best_model")
print(f"\n✅ 모델 저장 완료: {CONF['output_dir']}/best_model")

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/12457 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/12457 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/12457 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/499 [00:00<?, ? examples/s]

...

🚀 학습 시작...


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
200,0.671700,0.821098,0.774802,833098.000000,0.774581
400,0.674100,0.831118,0.815753,1664904.000000,0.772507
600,0.673300,0.814846,0.755387,2509369.000000,0.777479
800,0.643600,0.806955,0.797615,3339773.000000,0.778116
1000,0.651600,0.797188,0.785533,4182203.000000,0.781189
1200,0.637900,0.772395,0.755802,5018082.000000,0.788392
1400,0.631500,0.754182,0.741668,5843527.000000,0.793884
1600,0.522600,0.744097,0.618031,6683995.000000,0.798637
1800,0.526000,0.728678,0.614934,7519868.000000,0.802023
2000,0.528300,0.717974,0.594112,8350904.000000,0.804750


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/pyt


✅ 모델 저장 완료: ./results_solar_v2/best_model


## 7. 추론 및 평가

In [ ]:
from tqdm import tqdm
import re

def post_process(summary):
    """후처리"""
    summary = re.sub(r"\s+", " ", summary).strip()
    summary = summary.replace("### User:", "").replace("### Assistant:", "").strip()
    if not summary:
        return "요약을 생성할 수 없습니다."
    return summary

def generate_summary(model, tokenizer, dialogue, params):
    # 학습 시와 동일한 프롬프트 사용 (중요!)
    prompt = f"""### User:
다음 대화를 한국어로 요약하세요. #Person1#, #Person2# 등의 화자 표시는 그대로 유지하세요.

{dialogue}

### Assistant:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **params,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        summary = generated_text.split("### Assistant:\n")[1].strip()
    except:
        summary = generated_text
    
    return post_process(summary)

# 최적 추론 파라미터 (51.80점 설정)
INFERENCE_PARAMS = {
    "max_new_tokens": 150,
    "num_beams": 3,
    "early_stopping": True,
    "no_repeat_ngram_size": 0,
    "repetition_penalty": 1.2,
    "length_penalty": 1.3,
}

# Test 데이터 추론
test_df = pd.read_csv(os.path.join(CONF['data_path'], 'test.csv'))

print(f"🔄 추론 시작 (Test: {len(test_df)}개)")
print(f"   Params: {INFERENCE_PARAMS}")

model.eval()
summaries = []
for dialogue in tqdm(test_df['dialogue'], desc="Inference"):
    summary = generate_summary(model, tokenizer, dialogue, INFERENCE_PARAMS)
    summaries.append(summary)

# 결과 저장
submission = pd.DataFrame({
    'fname': test_df['fname'],
    'summary': summaries
})

output_path = "./prediction/submit_solar_v5.csv"
submission.to_csv(output_path, index=False)
print(f"\n✅ 저장 완료: {output_path}")

# 통계
lengths = [len(s.split()) for s in summaries]
print(f"\n📊 생성 요약 통계:")
print(f"  평균 길이: {sum(lengths)/len(lengths):.1f} words")
print(f"  최소/최대: {min(lengths)} / {max(lengths)} words")

🔄 추론 시작 (Test: 499개)
   Params: {'max_new_tokens': 150, 'num_beams': 3, 'early_stopping': True, 'no_repeat_ngram_size': 0, 'repetition_penalty': 1.2, 'length_penalty': 1.3}


Inference: 100%|██████████| 499/499 [13:12:59<00:00, 95.35s/it]   


✅ 저장 완료: ./prediction/submit_solar_v2.csv

📊 생성 요약 통계:
  평균 길이: 16.9 words
  최소/최대: 5 / 38 words


## 8. 추론 파라미터 최적화 실험 (재학습 없이 점수 향상)

In [ ]:
# v2 모델 추론 파라미터 최적화
# 목표: ROUGE-2 향상 (현재 42.19% → 42.5%+ 목표)

import pandas as pd
from tqdm import tqdm

# 실험할 파라미터 조합
EXPERIMENTS = {
    "exp1_simple_prompt": {
        "prompt_type": "simple",  # 프롬프트 단순화
        "params": {
            "max_new_tokens": 150,
            "num_beams": 3,
            "repetition_penalty": 1.2,
            "length_penalty": 1.3,
        }
    },
    "exp2_len_1.25": {
        "prompt_type": "simple",
        "params": {
            "max_new_tokens": 150,
            "num_beams": 3,
            "repetition_penalty": 1.2,
            "length_penalty": 1.25,  # 1.3 → 1.25
        }
    },
    "exp3_len_1.35": {
        "prompt_type": "simple",
        "params": {
            "max_new_tokens": 150,
            "num_beams": 3,
            "repetition_penalty": 1.2,
            "length_penalty": 1.35,  # 1.3 → 1.35
        }
    },
    "exp4_rep_1.15": {
        "prompt_type": "simple",
        "params": {
            "max_new_tokens": 150,
            "num_beams": 3,
            "repetition_penalty": 1.15,  # 1.2 → 1.15
            "length_penalty": 1.3,
        }
    },
    "exp5_rep_1.25": {
        "prompt_type": "simple",
        "params": {
            "max_new_tokens": 150,
            "num_beams": 3,
            "repetition_penalty": 1.25,  # 1.2 → 1.25
            "length_penalty": 1.3,
        }
    },
    "exp6_beam_4": {
        "prompt_type": "simple",
        "params": {
            "max_new_tokens": 150,
            "num_beams": 4,  # 3 → 4
            "repetition_penalty": 1.2,
            "length_penalty": 1.3,
        }
    },
}

def generate_with_prompt_type(model, tokenizer, dialogue, params, prompt_type="simple"):
    """프롬프트 타입별 생성"""
    if prompt_type == "simple":
        # 단순한 프롬프트 (v1 스타일에 가깝게)
        prompt = f"### User:\nSummarize the following dialogue:\n\n{dialogue}\n\n### Assistant:\n"
    else:
        # 원래 v2 프롬프트
        prompt = f"""### User:
다음 대화를 한국어로 요약하세요. #Person1#, #Person2# 등의 화자 표시는 그대로 유지하세요.

{dialogue}

### Assistant:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **params,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        summary = generated_text.split("### Assistant:\n")[1].strip()
    except:
        summary = generated_text
    
    return post_process(summary)

print("🔬 추론 파라미터 최적화 실험 시작")
print(f"총 {len(EXPERIMENTS)}개 실험 진행\n")

In [ ]:
# Dev 데이터로 빠른 검증 (시간 절약)
# Dev에서 가장 좋은 설정을 Test에 적용

from evaluate import load
rouge = load('rouge')

def evaluate_on_dev(model, tokenizer, dev_df, params, prompt_type="simple"):
    """Dev 데이터로 ROUGE 평가"""
    summaries = []
    references = []
    
    print(f"   Generating summaries on Dev ({len(dev_df)} samples)...")
    for idx, row in tqdm(dev_df.iterrows(), total=len(dev_df), leave=False):
        summary = generate_with_prompt_type(model, tokenizer, row['dialogue'], params, prompt_type)
        summaries.append(summary)
        references.append(row['summary'])
    
    # ROUGE 계산
    results = rouge.compute(predictions=summaries, references=references)
    
    return {
        'rouge1': results['rouge1'] * 100,
        'rouge2': results['rouge2'] * 100,
        'rougeL': results['rougeL'] * 100,
        'final': (results['rouge1'] + results['rouge2'] + results['rougeL']) / 3 * 100
    }

# Dev 데이터 로드
dev_df_eval = pd.read_csv(os.path.join(CONF['data_path'], 'dev.csv'))

print("📊 Dev 데이터 평가 시작\n")
results_summary = []

for exp_name, exp_config in EXPERIMENTS.items():
    print(f"🧪 {exp_name}")
    print(f"   Prompt: {exp_config['prompt_type']}")
    print(f"   Params: {exp_config['params']}")
    
    scores = evaluate_on_dev(
        model, 
        tokenizer, 
        dev_df_eval, 
        exp_config['params'],
        exp_config['prompt_type']
    )
    
    results_summary.append({
        'experiment': exp_name,
        'prompt': exp_config['prompt_type'],
        **exp_config['params'],
        **scores
    })
    
    print(f"   ✅ ROUGE-1: {scores['rouge1']:.2f}, ROUGE-2: {scores['rouge2']:.2f}, ROUGE-L: {scores['rougeL']:.2f}")
    print(f"   ⭐ Final: {scores['final']:.4f}\n")

# 결과 정리
results_df = pd.DataFrame(results_summary)
results_df = results_df.sort_values('final', ascending=False)

print("\n" + "="*80)
print("📈 실험 결과 요약 (Final Score 순)")
print("="*80)
print(results_df.to_string(index=False))
print("\n🏆 Best Setting:")
best = results_df.iloc[0]
print(f"   Experiment: {best['experiment']}")
print(f"   Final Score: {best['final']:.4f}")
print(f"   ROUGE-2: {best['rouge2']:.2f} (목표: 42.5+)")

In [ ]:
# Best 설정으로 Test 추론 및 제출 파일 생성

print("🚀 Best 설정으로 Test 데이터 추론 시작")
best_exp_name = results_df.iloc[0]['experiment']
best_config = EXPERIMENTS[best_exp_name]

print(f"\n📋 Best Setting: {best_exp_name}")
print(f"   Prompt Type: {best_config['prompt_type']}")
print(f"   Parameters: {best_config['params']}\n")

# Test 데이터 추론
test_df = pd.read_csv(os.path.join(CONF['data_path'], 'test.csv'))
test_summaries = []

model.eval()
for dialogue in tqdm(test_df['dialogue'], desc="Test Inference"):
    summary = generate_with_prompt_type(
        model, 
        tokenizer, 
        dialogue, 
        best_config['params'],
        best_config['prompt_type']
    )
    test_summaries.append(summary)

# 제출 파일 저장
submission = pd.DataFrame({
    'fname': test_df['fname'],
    'summary': test_summaries
})

output_path = f"./prediction/submit_solar_v2_optimized_{best_exp_name}.csv"
submission.to_csv(output_path, index=False)

print(f"\n✅ 저장 완료: {output_path}")

# 통계
lengths = [len(s.split()) for s in test_summaries]
print(f"\n📊 생성 요약 통계:")
print(f"   평균 길이: {sum(lengths)/len(lengths):.1f} words")
print(f"   최소/최대: {min(lengths)} / {max(lengths)} words")
print(f"\n🎯 예상 점수 (Dev 기준): {best['final']:.4f}")
print(f"   이전 최고: 51.8026")
print(f"   이전 v2: 51.7703")